# Exercice n°3 : Manipulation de données de neuroimagerie 🧠

Ce notebook est divisé en trois grandes sections :
1.  **IRM fonctionnelle (fMRI)** : Exploration de la connectivité avec **Nilearn**.
2.  **IRM de diffusion (dMRI)** : Reconstruction de tenseurs avec **Dipy**.
3.  **Électroencéphalographie (EEG)** : Traitement du signal avec **MNE-Python**.

L'objectif est de manipuler les structures de données fondamentales de la neuroimagerie (séries temporelles 4D, tenseurs de diffusion, signaux électriques).

**Consignes importantes pour l'évaluation automatique :**

1.  **Conformité** : Ne modifiez pas les noms des variables de résultat (ex: `q1_func_path`, `q11_raw`) donnés dans les lignes commentées.
2.  **Bibliothèques** : Utilisez `nilearn`, `dipy`, et `mne`.
3.  **Visualisation** : Les graphiques sont essentiels pour vérifier vos résultats. Des cellules de code commentées sont fournies pour vous guider.

Bon travail !

### Configuration
Exécutez cette cellule pour importer les modules nécessaires.

In [ ]:
# Assurez-vous d'avoir l'environnement nécessaire avant de commencer.

import numpy as np
import matplotlib.pyplot as plt
import mne

# Imports Nilearn
from nilearn import datasets, image, plotting
from nilearn.maskers import NiftiMapsMasker
from nilearn.connectome import ConnectivityMeasure

# Imports Dipy
from dipy.core.gradients import gradient_table
from dipy.io.image import load_nifti
from dipy.io.gradients import read_bvals_bvecs
from dipy.segment.mask import median_otsu
from dipy.reconst.dti import TensorModel

print("Bibliothèques importées avec succès.")

## Partie 1 : IRMf - Chargement et manipulation (10 points)

Nous allons travailler avec un sous-ensemble du jeu de données **ADHD200**.

### Question 1 : Chargement des données ADHD

Utilisez la fonction `datasets.fetch_adhd` de nilearn pour télécharger **1 seul sujet** (paramètre `n_subjects=1`).
Récupérez le chemin du fichier de l'image fonctionnelle (l'IRMf 4D) qui se trouve dans la liste `func` de l'objet retourné.

Stockez ce chemin (une chaîne de caractères) dans la variable **`q1_func_path`**.

In [ ]:
# Réponse 1

# Récupération du chemin de l'image fonctionnelle
q1_func_path = ...

print(q1_func_path)

### Question 2 : Création de l'image moyenne

Utilisez la fonction `image.mean_img` de nilearn sur le fichier `q1_func_path` pour générer une image 3D moyenne.
Stockez l'objet image résultant dans la variable **`q2_mean_img`**.

*Attente visuelle : Affichez cette image avec `plotting.plot_epi(q2_mean_img)`.*

In [ ]:
# Réponse 2

# Calcul de l'image moyenne
q2_mean_img = ...

# Visualisation
plotting.plot_epi(q2_mean_img, title="Image Moyenne")

## Partie 2 : IRMf - Extraction du signal (15 points)

Réduction de la dimensionnalité spatiale via un atlas.

### Question 3 : Extraction de séries temporelles (Atlas MSDL)

1. Téléchargez l'atlas probabiliste **MSDL** via `datasets.fetch_atlas_msdl()`.
2. Initialisez un objet `NiftiMapsMasker` avec : `maps_img` (l'atlas), `standardize=True`, `memory='nilearn_cache'`, `verbose=0`.
3. Appliquez `.fit_transform()` sur `q1_func_path`.

Stockez le tableau numpy résultant (taille : *temps* x *régions*) dans **`q3_time_series`**.

In [ ]:
# Réponse 3

q3_time_series = ...

print(f"Dimensions de la série temporelle : {q3_time_series.shape}")

## Partie 3 : IRMf - Connectivité (10 points)

Analyse des corrélations entre régions.

### Question 4 : Matrice de corrélation

Calculez la matrice de corrélation de Pearson à partir de `q3_time_series`.
Stockez la matrice (carrée, symétrique) dans **`q4_corr_matrix`**.

*Attente visuelle : Affichez la matrice avec `plotting.plot_matrix`.*

In [ ]:
# Réponse 4

q4_corr_matrix = ...

# Visualisation
plotting.plot_matrix(q4_corr_matrix, labels=atlas_msdl.labels, colorbar=True, vmax=1., vmin=-1., title="Matrice de Corrélation")

### Question 5 : Recherche de la connexion maximale

Trouvez l'indice de la région la plus fortement corrélée à la première région (indice 0), en excluant l'autocorrélation (diagonale).
Stockez cet indice (entier) dans **`q5_max_idx`**. Indice: utiliser `argmax` dans `numpy` sur la colonne `0` du connectome.

In [ ]:
# Réponse 5

q5_max_idx = ...

print(f"Indice de la région la plus connectée : {q5_max_idx}")

### Question 6 : Coordonnées ROI

Extrayez les coordonnées (x, y, z) de la région correspondant à l'indice `q5_max_idx` depuis l'attribut `region_coords` de l'atlas MSDL.
Stockez les coordonnées dans **`q6_coords`**.

In [ ]:
# Réponse 6

q6_coords = ...
print(f"Coordonnées de la région : {q6_coords}")

# Visualisation finale
plotting.plot_connectome(q4_corr_matrix, atlas_msdl.region_coords, edge_threshold='98%', title="Connectome MSDL (98% threshold)")

## Partie 4 : Imagerie de diffusion avec Dipy (32 points)

Dans cette section, nous allons utiliser **Dipy** pour reconstruire le tenseur de diffusion (DTI) à partir de données brutes. Le DTI permet d'estimer l'orientation des fibres de matière blanche.

Nous utiliserons le jeu de données `sherbrooke_3shell`. Il s'agit d'une acquisition HARDI, mais nous n'utiliserons ici que les données nécessaires pour le modèle tensoriel.

### Question 7 : Table de gradients (8 points)

Pour reconstruire le tenseur, nous avons besoin des images de diffusion et de la table de gradients (les directions dans lesquelles la diffusion a été mesurée).

1.  Utilisez `dipy.data.fetch_sherbrooke_3shell()` pour télécharger les données.
2.  Utilisez `dipy.data.get_fnames('sherbrooke_3shell')` pour obtenir les chemins des fichiers (`fraw`, `fbval`, `fbvec`).
3.  Chargez les valeurs `bvals` et `bvecs` avec `read_bvals_bvecs`.
4.  Créez la table de gradients `gtab` en utilisant la fonction `gradient_table` avec les `bvals` et `bvecs`.

Stockez l'objet table de gradients résultant dans la variable **`q7_gtab`**.

In [ ]:
# Réponse 7
import dipy.data

q7_gtab = ...

print(q7_gtab.info)

### Question 8 : Chargement et masquage (8 points)

Les images brutes contiennent souvent beaucoup de bruit en dehors du cerveau. Il est crucial de créer un masque.

1.  Chargez les données images Nifti (`fraw` obtenu à la question précédente) en utilisant `load_nifti`. Cela retourne `data` et `affine`.
2.  Utilisez la fonction `median_otsu` sur les données (`data`) pour segmenter le cerveau. Utilisez les paramètres `vol_idx=[0, 1]`, `median_radius=2`, `numpass=1`.
3.  Cette fonction retourne deux éléments : les données masquées et le masque binaire.

Stockez le **masque binaire** (tableau numpy) dans la variable **`q8_mask`**.

*Attente visuelle : Affichez la coupe axiale du milieu du masque.*

In [ ]:
# Réponse 8
masked_data, q8_mask = ...

# Visualisation
slice_idx = q8_mask.shape[2] // 2
plt.imshow(q8_mask[:, :, slice_idx], cmap='gray', origin='lower')
plt.title('Masque Binaire (Coupe Médiane)')
plt.show()

### Question 9 : Ajustement du modèle tensorial (8 points)

Nous allons maintenant calculer le modèle DTI voxel par voxel.

1.  Créez une instance du modèle tensoriel : `tenmodel = TensorModel(q7_gtab)`.
2.  Ajustez ce modèle aux données *masquées* (c'est-à-dire les données issues de `median_otsu` à la Q8, ou en appliquant `q8_mask` aux données brutes). Utilisez la méthode `fit` du modèle : `tenmodel.fit(data, mask=q8_mask)`.

Stockez l'objet résultant de l'ajustement (le "TensorFit") dans la variable **`q9_tenfit`**.

In [ ]:
# Réponse 9

q9_tenfit = ...

print("Modèle ajusté avec succès.")

### Question 10 : Fraction d'anisotropie (FA) (8 points)

La Fraction d'Anisotropie (FA) est une mesure scalaire entre 0 et 1 qui décrit le degré de directionnalité de la diffusion. Une FA proche de 1 indique une forte organisation des fibres (ex: corps calleux).

1.  Calculez la carte de FA à partir de l'objet `q9_tenfit` en utilisant son attribut `.fa`.
2.  Pour valider votre résultat, trouvez la valeur maximale de FA contenue dans cette carte.

Stockez cette valeur maximale (float) dans **`q10_max_fa`**.

*Attente visuelle : Affichez la carte de FA pour la coupe axiale médiane.*

In [ ]:
# Réponse 10

q10_max_fa = ...
print(f"Valeur maximale de FA : {q10_max_fa}")

# Visualisation
slice_idx = fa_map.shape[2] // 2
plt.imshow(fa_map[:, :, slice_idx], origin='lower', cmap='gray')
plt.title('Fraction d\'Anisotropie (FA)')
plt.colorbar()
plt.show()

## Partie 5 : Électroencéphalographie (EEG) avec MNE (33 points)

Dans cette dernière section, nous allons traiter des données EEG brutes pour extraire des potentiels évoqués (ERPs) liés à une stimulation auditive. Nous utiliserons le jeu de données classique "sample" de MNE.

### Question 11 : Chargement et sélection des canaux (6 points)

1.  Récupérez le chemin du jeu de données "sample" avec `mne.datasets.sample.data_path()`. Le fichier raw se trouve dans `MEG/sample/sample_audvis_raw.fif`.
2.  Chargez le fichier raw en utilisant `mne.io.read_raw_fif` avec l'option `preload=True` et stockez-le dans `q11_raw`.
3.  L'enregistrement contient des canaux MEG et EEG. Sélectionnez **uniquement les canaux EEG** en utilisant la méthode `.pick()` (ou `pick_types`) sur l'objet raw, après en avoir effectué une copie avec la méthode `.copy()`. Gardez uniquement les canaux de type 'eeg' (excluez meg, eog, stim).

Stockez l'objet Raw nettoyé contenant uniquement l'EEG dans la variable **`q11_raw_eeg`**.

In [ ]:
# Réponse 11
q11_raw = ...

q11_raw_eeg = ...

print(q11_raw_eeg.info)

### Question 12 : Filtrage (6 points)

Le signal EEG brut est souvent bruité et contient des dérives lentes.

1.  Créez une copie de vos données : `raw_filtered = q11_raw_eeg.copy()`.
2.  Appliquez un filtre passe-bande (bandpass filter) sur `raw_filtered` pour garder les fréquences entre **1 Hz et 40 Hz**.

Stockez l'objet Raw filtré dans la variable **`q12_raw_filtered`**.

In [ ]:
# Réponse 12

q12_raw_filtered = ...

### Question 13 : Création des époques (7 points)

Nous voulons découper le signal autour des stimulations auditives.

1.  Trouvez les événements dans les données brutes (`q11_raw`) en utilisant `mne.find_events`.
2.  Définissez l'identifiant de l'événement : `event_id = {'Auditory/Left': 1}`.
3.  Créez des époques (`Epochs`) autour de ces événements avec une fenêtre de temps allant de **-0.2s à 0.5s** (`tmin=-0.2`, `tmax=0.5`).
4.  Utilisez `baseline=(None, 0)` pour corriger la ligne de base et `preload=True`.

Stockez l'objet Epochs dans la variable **`q13_epochs`**.

In [ ]:
# Réponse 13

q13_epochs = ...

### Question 14 : Moyennage (potentiels évoqués) (7 points)

Pour voir la réponse cérébrale type, nous devons moyenner toutes les époques.

Calculez la moyenne des époques (`q13_epochs`) pour obtenir un objet `Evoked`.
Stockez cet objet dans la variable **`q14_evoked`**.

*Attente visuelle : Tracez les potentiels évoqués avec `q14_evoked.plot()`.*

In [ ]:
# Réponse 14

q14_evoked = ...

# Visualisation
q14_evoked.plot()

### Question 15 : Analyse de latence (7 points)

Le composant N100 est une réponse auditive négative qui apparaît environ 100ms après le stimulus.

1.  Utilisez la méthode `.get_peak()` sur votre objet `q14_evoked`.
2.  Cherchez le pic négatif (`mode='neg'`) dans la fenêtre de temps **0.08s à 0.12s**.
3.  La méthode retourne le nom du canal et la latence.

Stockez la **latence du pic** (en secondes, donc un float) dans la variable **`q15_peak_latency`**.

In [ ]:

ch_name, q15_peak_latency = ...

print(f"Le pic N100 a été trouvé à {q15_peak_latency} secondes sur le canal {ch_name}.")